# - TAXI NYC - MACHINE LEARNING - DATABRICKS

- Criando Tabela Silver a partir de Amostra de Dados Brutos:


In [0]:
df_bronze = spark.read.table("workspace.default.taxi_nyc_bronze")
df_bronze.write.mode("overwrite").saveAsTable("workspace.default.taxi_nyc_silver")

- Carregando a Tabela Silver e Iniciando a Limpeza:


In [0]:
df_silver = spark.read.table("workspace.default.taxi_nyc_silver")

- Corrigindo Tipos de Dados:


In [0]:
from pyspark.sql.functions import col, to_timestamp

df_silver_tipos_corrigidos = df_silver.withColumn("tpep_pickup_datetime", to_timestamp(col("tpep_pickup_datetime"))) \
                                      .withColumn("tpep_dropoff_datetime", to_timestamp(col("tpep_dropoff_datetime")))

df_silver_tipos_corrigidos.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- pickup_date: date (nullable = true)
 |-- pickup_day_of_week: long (nullable = true)
 |-- pickup_hour: long (nullable = true)
 |-- trip_duration_minutes: long (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- total_amount: double (nullable = true)



- Removendo Dados Nulos:

In [0]:
from pyspark.sql.functions import col

df_silver_filtrado = df_silver_tipos_corrigidos.filter(
    (col("passenger_count") > 0) &
    (col("trip_distance") > 0) &
    (col("total_amount") > 0)
)

- Adicionando Features (pickup_day_of_week,pickup_hour,trip_duration_minutes):

In [0]:
from pyspark.sql.functions import col, dayofweek, hour, round, unix_timestamp

df_silver_enriquecido = df_silver_filtrado.withColumn("pickup_day_of_week", dayofweek(col("tpep_pickup_datetime"))) \
                                          .withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))

duracao_em_segundos = unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))
df_silver_enriquecido = df_silver_enriquecido.withColumn("trip_duration_minutes", round(duracao_em_segundos / 60, 2))

- Salvando os Dados Limpos:

In [0]:
df_silver_enriquecido.write \
  .option("overwriteSchema", "true") \
  .mode("overwrite") \
  .saveAsTable("workspace.default.taxi_nyc_silver")

print("Tabela 'taxi_nyc_silver' atualizada com os dados limpos e enriquecidos.")

Tabela 'taxi_nyc_silver' atualizada com os dados limpos e enriquecidos.


" .option("overwriteSchema", "true") " Força o Delta Lake a ignorar o schema antigo da tabela e usar o novo schema.

- Criando a Camada Gold e Principais Métricas:

In [0]:

df_silver = spark.read.table("workspace.taxi_nyc_schema.taxi_nyc_silver")
from pyspark.sql.functions import count, sum, avg, round, col
# Agrupando os dados e calcular todas asmétricas
df_gold_completa = df_silver.groupBy("pickup_day_of_week", "pickup_hour") \
                            .agg(
                                count("*").alias("total_de_corridas"),
                                round(sum("total_amount"), 2).alias("faturamento_total"),
                                round(avg("total_amount"), 2).alias("faturamento_medio_por_corrida"),
                                round(avg("trip_distance"), 2).alias("distancia_media_percorrida"),
                                round(avg("trip_duration_minutes"), 2).alias("duracao_media_da_viagem_minutos"),
                                round(avg("passenger_count"), 2).alias("media_de_passageiros"),
                                round(avg(col("trip_distance") / (col("trip_duration_minutes") / 60)), 2).alias("velocidade_media_kmh")
                            ).filter(col("duracao_media_da_viagem_minutos") > 0) # Adicionado filtro para evitar divisão por zero

df_gold_completa.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.taxi_nyc_schema.taxi_nyc_gold")

print("Tabela Gold 'taxi_nyc_gold' criada e pronta para análise.")

Tabela Gold 'taxi_nyc_gold' criada e pronta para análise.


- Métricas Adicionadas: Total de Corridas, Faturamento Total, Faturamento Médio por Corrida, Distância Média Percorrida, Duração Média da Viagem em Minutos, Média de Passageiros e Velocidade Média! 


- Total de Corridas por Hora (Demanda):


In [0]:

%sql
SELECT
  pickup_hour as Hora_Embarque,
  SUM(total_de_corridas) AS volume_total_corridas
FROM
  workspace.taxi_nyc_schema.taxi_nyc_gold
GROUP BY
  pickup_hour
ORDER BY
  pickup_hour


Hora_Embarque,volume_total_corridas
0,2204
1,1477
2,998
3,658
4,476
5,647
6,1590
7,2919
8,3988
9,4509


In [0]:
%sql
SELECT
  pickup_hour,
  SUM(total_de_corridas) AS volume_total_corridas
FROM
  workspace.taxi_nyc_schema.taxi_nyc_gold
GROUP BY
  pickup_hour
ORDER BY
  pickup_hour

pickup_hour,volume_total_corridas
0,2204
1,1477
2,998
3,658
4,476
5,647
6,1590
7,2919
8,3988
9,4509


Databricks visualization. Run in Databricks to view.

 - Faturamento por Dia da Semana

In [0]:
%sql
SELECT
  CASE
    WHEN pickup_day_of_week = 1 THEN '1 - Domingo'
    WHEN pickup_day_of_week = 2 THEN '2 - Segunda'
    WHEN pickup_day_of_week = 3 THEN '3 - Terça'
    WHEN pickup_day_of_week = 4 THEN '4 - Quarta'
    WHEN pickup_day_of_week = 5 THEN '5 - Quinta'
    WHEN pickup_day_of_week = 6 THEN '6 - Sexta'
    WHEN pickup_day_of_week = 7 THEN '7 - Sábado'
  END as dia_da_semana,
  SUM(faturamento_total) as faturamento
FROM
  taxi_nyc_schema.taxi_nyc_gold -- <-- NOME DO SCHEMA ATUALIZADO AQUI
GROUP BY
  pickup_day_of_week
ORDER BY
  faturamento DESC;


dia_da_semana,faturamento
5 - Quinta,435108.76999999996
4 - Quarta,403499.92999999993
6 - Sexta,394801.41000000003
3 - Terça,383832.52
2 - Segunda,347733.4
7 - Sábado,333671.79999999993
1 - Domingo,290625.92


In [0]:
%sql
SELECT
  CASE
    WHEN pickup_day_of_week = 1 THEN '1 - Domingo'
    WHEN pickup_day_of_week = 2 THEN '2 - Segunda'
    WHEN pickup_day_of_week = 3 THEN '3 - Terça'
    WHEN pickup_day_of_week = 4 THEN '4 - Quarta'
    WHEN pickup_day_of_week = 5 THEN '5 - Quinta'
    WHEN pickup_day_of_week = 6 THEN '6 - Sexta'
    WHEN pickup_day_of_week = 7 THEN '7 - Sábado'
  END as dia_da_semana,
  SUM(faturamento_total) as faturamento
FROM
  taxi_nyc_schema.taxi_nyc_gold -- <-- NOME DO SCHEMA ATUALIZADO AQUI
GROUP BY
  pickup_day_of_week
ORDER BY
  faturamento DESC;

dia_da_semana,faturamento
5 - Quinta,435108.76999999996
4 - Quarta,403499.92999999993
6 - Sexta,394801.41000000003
3 - Terça,383832.52
2 - Segunda,347733.4
7 - Sábado,333671.79999999993
1 - Domingo,290625.92


Databricks visualization. Run in Databricks to view.

- Dias Úteis vs. Fim de Semana

In [0]:
%sql
SELECT
  CASE 
    WHEN pickup_day_of_week IN (2, 3, 4, 5, 6) THEN 'Dia Útil'
    ELSE 'Fim de Semana'
  END AS tipo_de_dia,
  pickup_hour as hora_embarque,
  SUM(total_de_corridas) as total_corridas
FROM
  taxi_nyc_schema.taxi_nyc_gold
GROUP BY
  tipo_de_dia,
  pickup_hour
ORDER BY
  tipo_de_dia DESC;

tipo_de_dia,hora_embarque,total_corridas
Fim de Semana,18,1525
Fim de Semana,14,1471
Fim de Semana,15,1489
Fim de Semana,0,1097
Fim de Semana,2,633
Fim de Semana,3,419
Fim de Semana,9,826
Fim de Semana,8,539
Fim de Semana,6,189
Fim de Semana,7,314


In [0]:
%sql
SELECT
  CASE 
    WHEN pickup_day_of_week IN (2, 3, 4, 5, 6) THEN 'Dia Útil'
    ELSE 'Fim de Semana'
  END AS tipo_de_dia,
  pickup_hour as hora_embarque,
  SUM(total_de_corridas) as total_corridas
FROM
  taxi_nyc_schema.taxi_nyc_gold
GROUP BY
  tipo_de_dia,
  pickup_hour
ORDER BY
  tipo_de_dia DESC;

tipo_de_dia,hora_embarque,total_corridas
Fim de Semana,18,1525
Fim de Semana,14,1471
Fim de Semana,15,1489
Fim de Semana,0,1097
Fim de Semana,2,633
Fim de Semana,3,419
Fim de Semana,9,826
Fim de Semana,8,539
Fim de Semana,6,189
Fim de Semana,7,314


Databricks visualization. Run in Databricks to view.

- Piores Horários para Dirigir

In [0]:
%sql
SELECT
  pickup_hour as hora_embarque,
  AVG(duracao_media_da_viagem_minutos) AS duracao_media_min,
  AVG(velocidade_media_kmh) AS velocidade_media
FROM
  taxi_nyc_schema.taxi_nyc_gold
GROUP BY
  pickup_hour 
ORDER BY
  velocidade_media ASC;


hora_embarque,duracao_media_min,velocidade_media
17,17.965714285714288,8.938571428571429
15,20.29714285714286,8.995714285714284
13,18.97714285714286,9.291428571428572
12,18.99285714285714,9.367142857142856
18,15.800000000000002,9.447142857142856
16,20.25857142857143,9.448571428571428
14,19.860000000000003,9.530000000000001
10,17.83857142857143,10.298571428571426
19,14.981428571428571,10.537142857142857
9,17.265714285714285,10.932857142857143


In [0]:
%sql
SELECT
  pickup_hour as hora_embarque,
  AVG(duracao_media_da_viagem_minutos) AS duracao_media_min,
  AVG(velocidade_media_kmh) AS velocidade_media
FROM
  taxi_nyc_schema.taxi_nyc_gold
GROUP BY
  pickup_hour 
ORDER BY
  velocidade_media ASC;


hora_embarque,duracao_media_min,velocidade_media
17,17.965714285714288,8.938571428571429
15,20.29714285714286,8.995714285714284
13,18.97714285714286,9.291428571428572
12,18.99285714285714,9.367142857142856
18,15.800000000000002,9.447142857142856
16,20.25857142857143,9.448571428571428
14,19.860000000000003,9.530000000000001
10,17.83857142857143,10.298571428571426
19,14.981428571428571,10.537142857142857
9,17.265714285714285,10.932857142857143


Databricks visualization. Run in Databricks to view.

- Heatmap da Operação

In [0]:
%sql
SELECT
  pickup_hour,
  CASE
    WHEN pickup_day_of_week = 1 THEN '1-Dom'
    WHEN pickup_day_of_week = 2 THEN '2-Seg'
    WHEN pickup_day_of_week = 3 THEN '3-Ter'
    WHEN pickup_day_of_week = 4 THEN '4-Qua'
    WHEN pickup_day_of_week = 5 THEN '5-Qui'
    WHEN pickup_day_of_week = 6 THEN '6-Sex'
    WHEN pickup_day_of_week = 7 THEN '7-Sab'
  END as dia_da_semana,
  total_de_corridas
FROM
  taxi_nyc_schema.taxi_nyc_gold

Databricks visualization. Run in Databricks to view.

pickup_hour,dia_da_semana,total_de_corridas
4,5-Qui,66
2,7-Sab,322
23,6-Sex,637
7,7-Sab,155
8,6-Sex,601
9,2-Seg,665
7,1-Dom,159
3,2-Seg,66
22,7-Sab,690
3,7-Sab,220


## MACHINE LEARNING (AMOSTA 100MIL CORRIDAS):

#### Ensinar um modelo a prever a coluna trip_duration_minutes com base em outras características da corrida com SCIKIT-LEARN

**MODELO DE ML:** Regressão Linear


- Carregando os Dados e Convertendo para Pandas.

In [0]:
spark_df = spark.table("taxi_nyc_schema.taxi_nyc_silver").select(
    "pickup_hour",
    "pickup_day_of_week",
    "trip_distance",
    "passenger_count",
    "trip_duration_minutes"
).na.drop()

pandas_df = spark_df.limit(100000).toPandas()

- Preparando os Features:

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

X = pandas_df[["pickup_hour", "pickup_day_of_week", "trip_distance", "passenger_count"]]
y = pandas_df["trip_duration_minutes"]

- Dividindo o Teste e Treinando o Modelo de Regressão Linear:

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lr_sklearn = LinearRegression()
lr_sklearn.fit(X_train, y_train)


LinearRegression()

- Previsão e Avaliação de Acurária:]

In [0]:
predictions = lr_sklearn.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)


- Resultado da Regressão Linear (AMOSTRA):


In [0]:
print(f"Modelo Scikit-learn treinado!")
print(f"RMSE: {rmse:.2f} minutos")
print(f"R²: {r2:.2%}")

Modelo Scikit-learn treinado!
RMSE: 15.71 minutos
R²: -0.38%


RMSE: Ele pode errar até 15.71min pra mais ou pra menos do que a duração da viagem!

Um R² negativo (o seu caso) significa que o seu modelo é PIOR do que um simples chute na média. Ele está ativamente fazendo previsões ruins. POR ISSO, vamos treinar OUTRO modelo e avaliar novamente:


**MODELO DE ML:** Random Forest

n_estimators=50 significa que ele usará 50 "árvores de decisão" para votar na melhor previsão.
n_jobs=-1 usa todos os processadores disponíveis para acelerar o treinamento.

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
rf_model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)

- Treindo Modelo:

In [0]:
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42)

- Previsão e Verificando Acurácia:

In [0]:
predictions_rf = rf_model.predict(X_test)

rmse_rf = np.sqrt(mean_squared_error(y_test, predictions_rf))
r2_rf = r2_score(y_test, predictions_rf)

- Resultado da Random Forest (AMOSTRA):

In [0]:

print("\n--- Resultado do Random Forest ---")
print(f"RMSE: {rmse_rf:.2f} minutos")
print(f"R²: {r2_rf:.2%}")


--- Resultado do Random Forest ---
RMSE: 9.09 minutos
R²: 66.39%


%md
RMSE: Ele pode errar até 9.09min pra mais ou pra menos do que a duração da viagem!

Um R² de 66.39% significa que ele pode acertar 2/3 das previsões! Bem melhor do que o modelo de regressão linear para esse caso!